# Custom Datasets

In [1]:
import random
from pprint import pprint

import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForSeq2Seq

from oumi.datasets import AlpacaDataset

### 1. Listing Supported Datasets

In [ ]:
from oumi.core.registry import REGISTRY, RegistryType


def list_datasets():
    """List all datasets in the registry."""
    for key, value in REGISTRY._registry.items():
        if key.registry_type == RegistryType.DATASET:
            print(key.name, "->", value.__name__)


list_datasets()

### 2. Loading Datasets

We load the alpaca dataset. Since multiple variants can be registered in the HuggingFace hub, by default we use `Dataset.default`.

In [ ]:
%%time

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
%%time

dataset = AlpacaDataset(tokenizer=tokenizer)

print(f"Using: {dataset.dataset_name}")

Alternatively, you can pass a custom HuggingFace hub identifier. You can find a list of supported datasets in `Dataset.supported_datasets`

In [ ]:
%%time

dataset = AlpacaDataset(dataset_name="yahma/alpaca-cleaned", tokenizer=tokenizer)

print(f"Using: {dataset.dataset_name}")

Under the hood, the dataset is downloaded from the HuggingFace hub, and cached in the `~/.cache/huggingface/datasets` directory.

When instantiating the class, the dataset is loaded in memory. This is acceptable with small datasets, but for larger datasets, we can either use `IterableDataset` for streaming batch from disk, or shard per worker rank (so that Memory // N_GPUs).

### 3. Iterating Over Dataset

In [ ]:
print(f"Number of examples: {len(dataset)}")

Given everything is loaded into memory, we can randomly access any row in the dataset:

In [ ]:
%%timeit

dataset[42]

We can iterate over the dataset to get the examples, either manually or using a DataLoader.

In [ ]:
%%time

# Manual iteration
[dataset[i] for i in range(len(dataset))];

In [ ]:
%%time

# With a pytorch data loader
loader = DataLoader(
    dataset, batch_size=1, num_workers=0, shuffle=False, collate_fn=lambda x: x
)
list(loader);

We can also use any library from the pytorch ecosystem, e.g. `torchtext`, `torchdata`, etc.

In [10]:
from torchdata.stateful_dataloader import StatefulDataLoader

loader = StatefulDataLoader(dataset, batch_size=1, num_workers=0, shuffle=False)

In [ ]:
loader.state_dict()

In [12]:
next(iter(loader));

### 4. Accessing Individual Examples

Alpaca is a "Supervised Finetuning Dataset". It contains instructions, user inputs, and model outputs. An SFT dataset has the following methods:

**Base Map Dataset**
```python
dataset[0] -> model inputs  # pytorch convention
dataset.raw(0) -> raw data  # oumi convention
```
**Base SFT Dataset**
```python
dataset.conversation(0) -> conversation # model independent
dataset.prompt(0) -> prompt  # dependends on the tokenizer
```

In [ ]:
# Raw data (pd.Series, dict), as defined by the dataset authors
dataset.raw(0)

In [ ]:
# Convert to standard Oumi SFT format (oumi.core.types.Conversation)
dataset.conversation(0).messages

In [ ]:
# Convert to model prompt (str). This include the model's chat temlate,
# EOS tokens for inference/generation, etc.
dataset.prompt(0)

In [ ]:
# What is fed to the model.forward (dict)
dataset[0]

### 5. Accessing the underlying data backend (for debugging only)

We currently use pd.DataFrame as a backend for the dataset. We can trivially use either HuggingFace `datasets`, or an `arrow` Table as a backend for the dataset.

In [ ]:
dataset.data.columns

In [ ]:
dataset.data.head()

In [ ]:
pip install -U -q matplotlib

In [ ]:
dataset.data[["instruction", "output", "input"]].map(len).hist();

### 6. Preprocessing

To customize preprocessing behavior, we can override the appropriate method:
- `BaseMapDataset.__getitem__` for fully custom behavior, multi-modal, multi-task, etc.
- `BaseMapDataset.transform` for custom preprocessing of standard datasets (inputs, labels)
- `BaseSftDataset.transform` To customize preprocessing, tokenization of a standard SFT dataset
- `BaseSftDataset.transform_conversation` To transform raw data row into oumi conversation

In [ ]:
dataset = AlpacaDataset(tokenizer=tokenizer)

print(f"Using: {dataset.dataset_name}")

In [ ]:
%%timeit

dataset.raw(0)

In [ ]:
%%timeit

idx = random.randint(0, len(dataset) - 1)
dataset.raw(idx)

In [ ]:
%%timeit

idx = random.randint(0, len(dataset) - 1)
dataset.conversation(idx)

In [ ]:
%%timeit

idx = random.randint(0, len(dataset) - 1)
dataset.prompt(idx)

In [ ]:
%%timeit

idx = random.randint(0, len(dataset) - 1)
dataset[idx]

## 7. Comparing with other datasets

In [27]:
import datasets

In [28]:
hf_dataset = datasets.load_dataset("tatsu-lab/alpaca")
hf_dataset = hf_dataset["train"]

#### Raw Random Access

In [ ]:
%%timeit

idx = random.randint(0, len(dataset) - 1)
dataset.raw(idx)  # use oumi dataset random access

In [ ]:
%%timeit

idx = random.randint(0, len(dataset) - 1)
hf_dataset[idx]  # use huggingface dataset random access

In [ ]:
%%timeit

idx = random.randint(0, len(dataset) - 1)
hf_dataset.data["text"][idx].as_py()  # directly access the arrow table.

#### With Tokenization

In [ ]:
hf_dataset.data["text"][0].as_py()

In [ ]:
%%timeit

# Arrow
text = hf_dataset.data["text"][0].as_py()
conversation = {"messages": [{"role": "user", "content": text}]}
dataset.tokenize(conversation)

In [ ]:
%%timeit

# HF Datasets
idx = random.randint(0, len(dataset) - 1)
dataset.tokenize(dataset.transform_conversation(hf_dataset[idx]))

In [ ]:
%%timeit

# Oumi dataset
idx = random.randint(0, len(dataset) - 1)
data = dataset.raw(idx).to_dict()
dataset.tokenize(dataset.transform_conversation(data))

In [ ]:
%%timeit

# Full Oumi pipeline
idx = random.randint(0, len(dataset) - 1)
dataset[idx]

### Iterate over dataset

In [ ]:
%%timeit

# Manual iteration
[dataset[i] for i in range(len(dataset))];

In [ ]:
%%timeit

# With a pytorch data loader
loader = DataLoader(dataset, batch_size=1, num_workers=0, shuffle=False)
list(loader);

In [ ]:
pip install -U -q line_profiler memory_profiler

In [40]:
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
%prun [dataset[i] for i in range(len(dataset))];

In [ ]:
%lprun -f dataset.__getitem__ [dataset[i] for i in range(len(dataset))];

In [ ]:
%memit [dataset[i] for i in range(len(dataset))]

## 8. Benchmark with Model Forward Pass

In [ ]:
%%time

model = AutoModelForCausalLM.from_pretrained("gpt2")

In [ ]:
%%time

dataset = AlpacaDataset(tokenizer=tokenizer)
collator_fn = DataCollatorForSeq2Seq(tokenizer=tokenizer, return_tensors="pt")

loader = DataLoader(
    dataset, batch_size=3, num_workers=0, shuffle=False, collate_fn=collator_fn
)

In [ ]:
%%timeit

# Including pre-processing
with torch.no_grad():
    batch = next(iter(loader))
    model.forward(**batch)

In [47]:
fixed_batch = next(iter(loader))

In [ ]:
%%timeit

# Excluding pre-processing
with torch.no_grad():
    model.forward(**fixed_batch)

## 9. Testing Different Tokenizers

In [49]:
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token
gpt_tokenizer.chat_template = phi_tokenizer.chat_template
llama_tokenizer = None
try:
    llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
except Exception as e:
    print("Skipping Llama 3.2, since you don't have access to it.")
    print(e)

In [ ]:
dataset = AlpacaDataset()

pprint(dataset.conversation(0).messages)

In [ ]:
dataset._tokenizer = gpt_tokenizer
pprint(dataset.prompt(0))

In [ ]:
dataset._tokenizer = phi_tokenizer
pprint(dataset.prompt(0))

In [ ]:
if llama_tokenizer is not None:
    dataset._tokenizer = llama_tokenizer
    pprint(dataset.prompt(0))

## 10. Test Packing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.pad_token = tokenizer.eos_token
dataset = AlpacaDataset(tokenizer=tokenizer)

In [ ]:
from oumi.core.datasets.pretraining_async_text_dataset import (
    PretrainingAsyncTextDataset,
)

In [ ]:
dataset = PretrainingAsyncTextDataset(
    tokenizer,
    dataset,
    dataset_text_field=None,
    formatting_func=lambda x: x,
    pretokenized=True,
)

In [ ]:
next(iter(dataset))